## 0. Scrape Data

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
# adapted from https://github.com/CoryKjar/UFO-Analysis/blob/main/nuforc_Scrape.py

links = []
dataframes = []
response = requests.get('https://nuforc.org/webreports/ndxevent.html')
soup = BeautifulSoup(response.content, "lxml")
table = soup.find('table')

for row in tqdm(table.find_all('a', href=True), desc="Getting links...", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt}"):
    links.append(row['href'])

for link in tqdm(links, desc="Downloading...", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt}"):
    url = f'https://nuforc.org/webreports/{link}'
    data = pd.read_html(url)[0] 
    tempdf = pd.DataFrame(data)
    dataframes.append(tempdf)
df = pd.concat(dataframes)

df.to_csv(r'data.csv', index=False, sep=';')
print('done')


Getting links...: 100%|██████████████████████████████████████████████████████████████████████████████████████ | 978/978
Downloading...: 100%|████████████████████████████████████████████████████████████████████████████████████████ | 978/978


done


## 1. Clean Data

In [3]:
df = pd.read_csv('data.csv', delimiter=';')
df

,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,5/19/23 01:49,Harrisburg East of I5,OR,USA,Light,Approx. 3-4 min.,9 lights in straight line,5/19/23,NaN
1,5/18/23 19:40,Kippens,NF,Canada,Circle,5 minutes,Looked out window and saw the sun catching on ...,5/19/23,Yes
2,5/18/23 13:27,Hurricane,UT,USA,Cigar,10 minutes,Saw what looked like a long cigar shaped objec...,5/19/23,NaN
3,5/17/23 23:20,Sharpsville,IN,USA,Oval,1 hour,Oval like object over town no brightness,5/19/23,NaN
4,5/17/23 23:00,Tadepalligudem,Andhra Pradesh,India,Cube,5 minutes,We are watching the night sky suddenly there i...,5/19/23,Yes
...,...,...,...,...,...,...,...,...,...
144446,2010,Amherst,VA,USA,NaN,NaN,This is a still shot recently taken from video...,3/18/14,NaN
144447,NaN,Graham,WA,USA,Disk,2 minutes,Driving west on 224th looked right saw large c...,6/4/14,NaN
144448,NaN,NaN,NaN,USA,NaN,NaN,((HOAX??)) I work as a security guard in march...,2/22/17,NaN
144449,NaN,Subotica,NaN,NaN,NaN,NaN,Objects was moving very fast on this about pos...,8/5/12,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144451 entries, 0 to 144450
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date / Time  144350 non-null  object
 1   City         143926 non-null  object
 2   State        134632 non-null  object
 3   Country      144060 non-null  object
 4   Shape        138074 non-null  object
 5   Duration     137350 non-null  object
 6   Summary      144378 non-null  object
 7   Posted       144448 non-null  object
 8   Images       2342 non-null    object
dtypes: object(9)
memory usage: 9.9+ MB


In [5]:
print('num rows with no nulls:', (df
                                  .notnull()
                                  .all(axis=1)
                                  .sum()))
print('excluding image col:', (df
                               .loc[:, df.columns[:-1]]
                               .notnull()
                               .all(axis=1)
                               .sum()))

num rows with no nulls: 1996
excluding image col: 125065


In [6]:
# unique values
for col in df.columns:
    print(col, len(df[col].unique()))

Date / Time 123616
City 29456
State 359
Country 445
Shape 41
Duration 14653
Summary 141606
Posted 621
Images 2


We have 9 columns, all strings & all with some missing values. It appears that the naming conventions used for states & countries are inconsistent and will require cleaning and conversion to more appropriate data types.

1. Date / Time - datetime
2. City - Categorical
3. State - Categorical
4. Country - Categorical
5. Shape - Categorical
6. Duration - datetime/int?
7. Summary - string
8. Posted - datetime
9. Images - boolean

Starting with country - there are 195-197 countries and we have 444 due to aliases

In [7]:
# load list of countries from text file
with open('countries.txt', 'r') as f:
    contents = f.read()
 
# split on newline
countries = contents.split('\n')

# some countries have multiple names on the same line split by pipe character
# can explode these into a list for matching
alt_countries = []
for c in countries:
    alt_names = c.split('|')
    if len(alt_names) > 1:
        for n in alt_names:
            alt_countries.append(n)
    else:
        alt_countries.append(alt_names[0])
        

In [8]:
countries[:3]

['afghanistan', 'albania', 'algeria']

In [9]:
# use fuzzy matching to match misspelt entries
from fuzzywuzzy import process

def correct_country_names(country_name):
    correct_names = alt_countries
    match, score = process.extract(country_name, correct_names)[0]
    if score >= 70:
        return match
    else:
        return country_name

df['correct_country'] = df['Country'].fillna('NONE').apply(correct_country_names)

In [ ]:
country_mappings = {}

for c in countries:
    alternate_names = c.split('|')
    # use the shortest name
    name_to_use = min(alternate_names, key=len)
    for d in alternate_names:
        country_mappings[d] = name_to_use

df['matched_country'] = df['matched_country'].str.replace('|'.join(country_mappings.keys()), lambda x: country_mappings[x.group()])

In [53]:
# save file as this was very slow to process
#df.to_csv('clean_data.csv')
df = pd.read_csv('clean_data.csv', index_col=0)

In [54]:
df.correct_country.value_counts(1)

usa                       0.890538
canada                    0.040197
united kingdom            0.022377
australia                 0.006357
india                     0.003231
                            ...   
cabo verde                0.000007
Caribbean (Grand Turk)    0.000007
guinea                    0.000007
Decalb                    0.000007
unknown/at sea            0.000007
Name: correct_country, Length: 272, dtype: float64

USA makes up 89% of sightings recorded in database.

In [55]:
# rename columns
df.columns = df.columns.str.replace('[^a-zA-Z0-9]+', '', regex=True)
df.columns = df.columns.str.lower()
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration', 'summary',
       'posted', 'images', 'correctcountry'],
      dtype='object')

In [56]:
# convert to datetime
df.datetime

0         4/8/23 19:30
1         4/8/23 18:45
2         4/8/23 05:39
3         4/7/23 22:15
4         4/7/23 21:55
              ...     
143935             NaN
143936             NaN
143937             NaN
143938             NaN
143939             NaN
Name: datetime, Length: 143940, dtype: object

In [57]:
df['datetime_fixed'] = pd.to_datetime(df.datetime, format='%m/%d/%y %H:%M', errors='coerce')

In [67]:
df.datetime_fixed.sort_values(ascending=False).head(10) # are we getting 2068 instead of 1968?

141214   2068-12-25 03:00:00
141216   2068-12-20 02:00:00
141215   2068-12-20 02:00:00
141217   2068-12-15 23:00:00
141218   2068-12-15 21:00:00
141219   2068-12-15 20:00:00
141220   2068-12-15 17:30:00
141221   2068-12-12 03:00:00
141222   2068-12-10 00:30:00
141223   2068-12-03 02:00:00
Name: datetime_fixed, dtype: datetime64[ns]

In [68]:
df.datetime[141214]

'12/25/68 03:00'

In [74]:
# look for values occurring in the future & subtract 100 years
import datetime
mask = df.datetime_fixed > datetime.datetime(2023, 5, 1)
df.loc[mask, 'datetime_fixed'] -= pd.DateOffset(years=100)

In [76]:
df.sort_values('datetime_fixed') # new earliest sighting is 1925  

,datetime,city,state,country,shape,duration,summary,posted,images,correctcountry,datetime_fixed
143525,12/28/25 18:00,Atkinson (6 miles north of),IL,USA,Disk,1 minute,Young boy witnesses disc in sky above Illinois...,5/11/05,NaN,usa,1925-12-28 18:00:00
143524,7/12/28 10:45,PASADENA,CA,USA,Light,10 minutes,Cluster of lights in So. California,1/19/21,NaN,usa,1928-07-12 10:45:00
143523,1/28/29 15:41,Santa Teresa,NM,USA,Circle,5 mins,large craft with other smaller crafts moving f...,1/19/21,NaN,usa,1929-01-28 15:41:00
143522,7/5/29 14:00,Buchanan (or Burns),OR,USA,Disk,1 minute,"we were traveling east of burns,clmbing up thr...",8/16/02,NaN,usa,1929-07-05 14:00:00
143521,6/12/30 22:00,Charlotte,NC,USA,Orb,2 minutes,Orb discharges after moving slowly from creek ...,1/19/21,NaN,usa,1930-06-12 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...
143935,NaN,Kelowna (Canada),BC,Canada,NaN,NaN,"They were close to my home, hope they come soo...",7/3/13,NaN,canada,NaT
143936,NaN,Rule,TX,USA,Unknown,15 minutes,Unusual sounds and starlight distortion defini...,8/19/12,NaN,usa,NaT
143937,NaN,Gulf Breeze,FL,USA,NaN,NaN,The gulf breeze florida sightings were a AC130...,8/5/12,NaN,usa,NaT
143938,NaN,Richland,WA,USA,Orb,10 min,Multiple sightings,12/20/12,NaN,usa,NaT


In [78]:
df.duration # already can see misspellings...this will be difficult to parse

0          96 minutes
1          20 secinds
2         3-5 minutes
3              1 hour
4               2 min
             ...     
143935            NaN
143936     15 minutes
143937            NaN
143938         10 min
143939            NaN
Name: duration, Length: 143940, dtype: object

In [109]:
# extract number first
import re
import numpy as np

def extract_duration_number(row):
    try: 
        n = re.search('\d+', row['duration'])[0]
        n = int(n)
    except:
        n = np.nan
    return n
        
df['duration_num'] = df.apply(extract_duration_number, axis=1)

In [110]:
df.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143940 entries, 0 to 143939
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   datetime        143839 non-null  object        
 1   city            143422 non-null  object        
 2   state           134143 non-null  object        
 3   country         143562 non-null  object        
 4   shape           137568 non-null  object        
 5   duration        136868 non-null  object        
 6   summary         143867 non-null  object        
 7   posted          143937 non-null  object        
 8   images          2203 non-null    object        
 9   correctcountry  143940 non-null  object        
 10  datetime_fixed  141837 non-null  datetime64[ns]
 11  duration_num    126581 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 18.3+ MB


In [105]:
re.search('\d+', df.duration[0])[0]

'96'

In [117]:

def identify_duration_unit(duration_text):
    units = ('hours', 'minutes', 'seconds', 'days', 'min')
    match, score = process.extract(duration_text, units)[0]
    if score >= 40:
        return match
    else:
        return 'NONE'

df['duration_unit'] = df['duration'].fillna('NONE').apply(identify_duration_unit)
df.loc[df.duration_unit=='min'] = 'minutes'

In [120]:
# several types of duration are left, including single times like 2:05 or 15:45 
# (could represent time it appeared or hour/min or min/sec)
# time ranges; 12:05 - 13:05 
# and random text descriptions

df.loc[df.duration_unit=='NONE']

,datetime,city,state,country,shape,duration,summary,posted,images,correctcountry,datetime_fixed,duration_num,duration_unit
38,3/31/23 22:49,NaN,NaN,USA,Circle,17:20,I was looking on google maps and saw a strange...,4/9/23,NaN,usa,2023-03-31 22:49:00,17.0,NONE
135,3/15/23 21:26,Bellevue,WA,USA,Circle,Still visible,"Glowing circle, stationary",4/9/23,NaN,usa,2023-03-15 21:26:00,NaN,NONE
152,3/12/23 21:40,Oxnard,CA,USA,Unknown,9:40pm - 10:30pm,850' AGL directly overhead @ 65 mph air speed ...,4/9/23,NaN,usa,2023-03-12 21:40:00,9.0,NONE
160,3/11/23 19:35,East Hartford,CT,USA,Sphere,2:05,I got off the bus and noticed 2 spherical red ...,4/9/23,Yes,usa,2023-03-11 19:35:00,2.0,NONE
213,3/5/23 14:07,Glenhaven,NaN,NaN,Cube,Wolfgang Johnny,Like a clear beach ball with the cube in it,3/6/23,NaN,lebanon,2023-03-05 14:07:00,NaN,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143829,NaN,Santa Maria,CA,USA,Circle,19:00,Watching Sunday or Monday night football went ...,1/31/20,NaN,usa,NaT,19.0,NONE
143852,NaN,Rockville,MD,USA,Other,0:00-3:00,intermittent blinking (.5 of a second) objects...,7/31/21,NaN,usa,NaT,0.0,NONE
143877,NaN,Poca,WV,USA,Circle,All night all summer,"I've reported this 3 times ,but it's never pos...",12/23/20,NaN,usa,NaT,NaN,NONE
143906,?,Marysville,WA,USA,NaN,?,"Look, I was telling my two friends from My acr...",8/24/10,NaN,usa,NaT,NaN,NONE


In [122]:
# duration number will be invalid for these entries
df.loc[df.duration_unit=='NONE', 'duration_num'] = np.nan

In [206]:
pattern = '(\d{1,2}:\d{2}?[ap]m)'
test = '9:40pm - 10:30pm'
matches = re.findall(pattern, test)
matches

['9:40pm', '10:30pm']

In [212]:
def extract_duration_times_with_no_units(duration_text):
    
    pattern = '(\d{1,2}:\d{2}[ap]m)'
    if type(duration_text) != str:
        duration_text = str(duration_text)
    matches = re.findall(pattern, duration_text)
    if matches:
        return tuple(matches)
    else:
        pattern = '(\d{1,2}:\d{2})'
        matches = re.findall(pattern, duration_text)
    if matches:
        return tuple(matches)
    else:
        return np.nan


In [218]:
time_matches = df.loc[df.duration_unit=='NONE'].duration.apply(extract_duration_times_with_no_units)
time_matches

38                 (17:20,)
135                     NaN
152       (9:40pm, 10:30pm)
160                 (2:05,)
213                     NaN
                ...        
143829             (19:00,)
143852         (0:00, 3:00)
143877                  NaN
143906                  NaN
143907                  NaN
Name: duration, Length: 7138, dtype: object

In [226]:
first_time = time_matches.apply(lambda x: x[0] if x==x else np.nan)
first_time.name = 'duration_time_1'
second_time = time_matches.apply(lambda x: x[1] if (x==x and len(x)>1) else np.nan)
second_time.name = 'duration_time_2'
df = pd.concat([df, first_time, second_time], axis=1)

,datetime,city,state,country,shape,duration,summary,posted,images,correctcountry,datetime_fixed,duration_num,duration_unit,duration_time_1,duration_time_2
38,3/31/23 22:49,NaN,NaN,USA,Circle,17:20,I was looking on google maps and saw a strange...,4/9/23,NaN,usa,2023-03-31 22:49:00,NaN,NONE,17:20,NaN
135,3/15/23 21:26,Bellevue,WA,USA,Circle,Still visible,"Glowing circle, stationary",4/9/23,NaN,usa,2023-03-15 21:26:00,NaN,NONE,NaN,NaN
152,3/12/23 21:40,Oxnard,CA,USA,Unknown,9:40pm - 10:30pm,850' AGL directly overhead @ 65 mph air speed ...,4/9/23,NaN,usa,2023-03-12 21:40:00,NaN,NONE,9:40pm,10:30pm
160,3/11/23 19:35,East Hartford,CT,USA,Sphere,2:05,I got off the bus and noticed 2 spherical red ...,4/9/23,Yes,usa,2023-03-11 19:35:00,NaN,NONE,2:05,NaN
213,3/5/23 14:07,Glenhaven,NaN,NaN,Cube,Wolfgang Johnny,Like a clear beach ball with the cube in it,3/6/23,NaN,lebanon,2023-03-05 14:07:00,NaN,NONE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143829,NaN,Santa Maria,CA,USA,Circle,19:00,Watching Sunday or Monday night football went ...,1/31/20,NaN,usa,NaT,NaN,NONE,19:00,NaN
143852,NaN,Rockville,MD,USA,Other,0:00-3:00,intermittent blinking (.5 of a second) objects...,7/31/21,NaN,usa,NaT,NaN,NONE,0:00,3:00
143877,NaN,Poca,WV,USA,Circle,All night all summer,"I've reported this 3 times ,but it's never pos...",12/23/20,NaN,usa,NaT,NaN,NONE,NaN,NaN
143906,?,Marysville,WA,USA,NaN,?,"Look, I was telling my two friends from My acr...",8/24/10,NaN,usa,NaT,NaN,NONE,NaN,NaN


In [10]:
geocoded_df = pd.read_csv('nuforc_reports.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'nuforc_reports.csv'

In [ ]:
geocoded_df 

In [ ]:
import geopandas as gpd

test_df = pd.DataFrame({'city': ['A', 'B', 'C'], 
                   'latitude': [41.231, 42.234, 41.76],
                   'longitude': [-88.3, -94.2, -75.1], 
                   'category': ['small', 'medium','small'],
                   'value' : [32, 45, 54]})

gdf = gpd.GeoDataFrame(
    test_df, geometry=gpd.points_from_xy(test_df.longitude, test_df.latitude))

print(gdf)

In [246]:
gdf = gpd.GeoDataFrame(
    geocoded_df, geometry=gpd.points_from_xy(geocoded_df.city_longitude, geocoded_df.city_latitude))

In [247]:
import chart_studio.graph_objs as go

# min year in your dataset
year = 1998

# your color-scale
scl = [[0.0, '#ffffff'],[0.2, '#b4a8ce'],[0.4, '#8573a9'],
       [0.6, '#7159a3'],[0.8, '#5732a1'],[1.0, '#2c0579']] # purples

data_slider = []
for year in df['years'].unique():
    df_segmented =  df[(df['years']== year)]

    for col in df_segmented.columns:
        df_segmented[col] = df_segmented[col].astype(str)

    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['state'],
                        z=df_segmented['sightings'].astype(float),
                        locationmode='USA-states',
                        colorscale = scl,
                        colorbar= {'title':'# Sightings'})

    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1998))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

layout = dict(title ='UFO Sightings by State Since 1998', geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)

fig = dict(data=data_slider, layout=layout)
periscope.plotly(fig)

ModuleNotFoundError: No module named 'chart_studio.graph_objs'

In [248]:
gdf

,summary,country,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,geometry
0,MADAR Node 100,USA,Mountlake Terrace,WA,2019-06-23T18:53:00,NaN,NaN,Occurred : 6/23/2019 18:53 (Entered as : 06/2...,http://www.nuforc.org/webreports/reports/147/S...,MADAR Node 100,2019-06-27T00:00:00,47.794100,-122.306600,POINT (-122.30660 47.79410)
1,Steady flashing object with three lights hover...,USA,Hamden,CT,2019-06-23T20:00:00,light,5 hours,Occurred : 6/23/2019 20:00 (Entered as : 6/23...,http://www.nuforc.org/webreports/reports/147/S...,Steady flashing object with three lights hover...,2019-06-27T00:00:00,41.373941,-72.921325,POINT (-72.92132 41.37394)
2,"Group of several orange lights, seemingly circ...",USA,Charlottesville,VA,2019-06-20T23:28:00,circle,15 seconds,Occurred : 6/20/2019 23:28 (Entered as : 06/2...,http://www.nuforc.org/webreports/reports/146/S...,"Group of several orange lights, seemingly circ...",2019-06-27T00:00:00,38.055968,-78.494482,POINT (-78.49448 38.05597)
3,Dropped in flashed a few times and shot off 5 ...,USA,Lincoln Park,MI,2019-06-21T00:00:00,light,2 minutes,Occurred : 6/21/2019 00:00 (Entered as : 06/2...,http://www.nuforc.org/webreports/reports/146/S...,Dropped in flashed a few times and shot off 5 ...,2019-06-27T00:00:00,42.238500,-83.178300,POINT (-83.17830 42.23850)
4,"Location: While traveling in a TGV, from Lille...",France,Douai (France),NaN,2019-06-07T20:00:00,cigar,5 minutes,Occurred : 6/7/2019 20:00 (Entered as : 06/07...,http://www.nuforc.org/webreports/reports/146/S...,"Location: While traveling in a TGV, from Lill...",2019-06-27T00:00:00,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141256,See craft every evening when I get out of work...,USA,Branford,CT,2021-12-19T23:00:00,circle,NaN,Occurred : 12/19/2021 23:00 (Entered as : 12/...,http://www.nuforc.org/webreports/reports/165/S...,See craft every evening when I get out of work...,2021-12-19T00:00:00,41.277600,-72.811500,POINT (-72.81150 41.27760)
141257,MADAR Node 70,USA,St Louis,MO,2022-03-09T00:48:00,NaN,NaN,Occurred : 3/9/2022 00:48 (Entered as : 03/09...,http://www.nuforc.org/webreports/reports/167/S...,MADAR Node 70 \n \n,2022-04-22T00:00:00,38.626989,-90.307029,POINT (-90.30703 38.62699)
141258,A very small white light hovering above the cl...,USA,Batavia,IL,2013-04-30T02:19:00,circle,45 seconds,Occurred : 4/30/2013 02:19 (Entered as : 04/3...,http://www.nuforc.org/webreports/reports/097/S...,A very small white light hovering above the cl...,2013-05-15T00:00:00,41.847800,-88.307500,POINT (-88.30750 41.84780)
141259,I was young. You know what? It was pretty fuck...,USA,Liberty Lake,WA,1995-07-21T09:00:00,circle,1/12,Occurred : 7/21/1995 09:00 (Entered as : 07/2...,http://www.nuforc.org/webreports/reports/151/S...,I was young. You know what? It was pretty ((...,2019-12-19T00:00:00,47.651700,-117.083800,POINT (-117.08380 47.65170)


In [250]:
import plotly.express as px
import geopandas as gpd

fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.geometry,
                           locations=gdf.country,
                           color="Joly",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="open-street-map",
                           zoom=8.5)
fig.show()

ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['summary', 'country', 'city', 'state', 'date_time', 'shape', 'duration', 'stats', 'report_link', 'text', 'posted', 'city_latitude', 'city_longitude', 'geometry'] but received: Joly